## Data Science Internship
#### Regressão
##### Autor: Pedro Henrique Braga Lisboa

#### Sumário
----
* [Escolha do modelo](#set_model)
* [Treinamento](#training)
* [Resultados](#results)

In [ ]:
import os
import pandas as pd
import numpy as np
import joblib as jbl
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn import cross_decomposition
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import TimeSeriesSplit, cross_validate

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

%matplotlib inline
matplotlib.rc('font', family='Arial')
sns.set_style("whitegrid")

In [ ]:
dataset = pd.read_csv("qualidade_do_ar_preenchida.csv")
dataset.drop(["Date", "Time", "Weekday"], axis=1, inplace=True)
outer_n_splits=2
inner_n_splits=3

inner_tscv = TimeSeriesSplit(n_splits=inner_n_splits)
outer_tscv = TimeSeriesSplit(n_splits=outer_n_splits)

### Escolha do modelo <a name="set_model"></a>

#### Foi observado durante a análise exploratória que algumas variáveis que serão usadas para prever a concentração e CO também são fortemente correlacionadas entre sí (e.g. C6H6(GT) e PT08.S2(NMHC)). Um termo de regularização de norma $L_2$ adicionado à função custo pode impedir o aumento demasiado dos coeficientes de variáveis correlacionadas.   

#### No entanto, existe a possibilidade de algumas variáveis selecionadas serem redundantes para prever a concentração de CO. Um termo de regularização de norma $L_1$ pode ser adicionado à função custo do modelo para buscar obter um conjunto esparso de coeficientes, eliminando a contribuição de algumas variáveis. 

#### Dessa forma, a regressão será feita com o modelo ElasticNet, combinando as duas regularizações. Um procedimento de validação para avaliação dos resultados a partir do Coeficiente de Determinação $R^2$ (ajustado para o caso multilinear) e a raiz do Erro Médio Quadrático(RMSE). 
#### Um procedimento de validação cruzada com duas rodadas aninhadas será usado para evitar a polarização dos resultados. A seleção do melhor modelo será feita no loop interno do procedimento, e a avaliação do modelo no loop externo. 

### Treinamento <a name="training"></a>
---

In [ ]:
regression_vars = ['C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)',
                  'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)']
pred_var = "PT08.S1(CO)"

def adj_r2(y_true, y_pred, *r2_args):
    """ Score R2 ajustado para o caso multivariável"""
    
    r2 = r2_score(y_true, y_pred, *r2_args)
    n = dataset.shape[0]
    p = dataset.shape[1]
    
    return 1-(1-r2)*(n-1)/(n-p-1)

def build_pipeline(model):
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('regressor', model))
    return Pipeline(estimators)

def eval_model(pipeline,verbose=0):
    return cross_validate(pipeline,
                        dataset.loc[:, regression_vars],
                        dataset.loc[:, pred_var],
                        scoring={"neg_mean_squared_error":"neg_mean_squared_error",
                                 "adj_r2":make_scorer(adj_r2)}, 
                        cv = outer_tscv,
                        return_estimator=True,
                        verbose=verbose)

model = build_pipeline(linear_model.ElasticNetCV(cv=inner_tscv))
results = eval_model(model)
results["test_neg_mean_squared_error"] = np.sqrt(results["test_neg_mean_squared_error"]*(-1))

### Resultados <a name="results"></a>
---

In [ ]:
print("Mean Adj R2: " + '%.2f' % results['test_adj_r2'].mean() 
      + " +-" + ' %.2f' % results['test_adj_r2'].std())
print("Mean RMSE: " + '%.2f' % results['test_neg_mean_squared_error'].mean() 
      + " +-" + ' %.2f' % results['test_neg_mean_squared_error'].std())

pd.DataFrame({"Adj R2":results['test_adj_r2'],
              "RMSE":results['test_neg_mean_squared_error']}, index=["Fold 1", "Fold 2"])

#### Os resultados apontam para um bom fit do modelo. Com um coeficiente de determinação em média de 80%, a maior parte da variação da variável dependente é explicada pelo modelo. 

#### Observando o RMSE, pode-se ter uma ideia do quão próximo as previsões do modelo se aproximam do alvo. Considerando a faixa de valores da variável de predição [640, 2040], os valores do RMSE e os gráficos abaixo, o modelo é capaz de entregar boas estimativas.

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(15,5))
for i, (_, test) in enumerate(outer_tscv.split(dataset)):
    X = dataset[regression_vars].iloc[test]
    y_true = dataset[pred_var].iloc[test]
    y_pred = results['estimator'][i].predict(X)
    
    x=np.linspace(y_pred.min(),y_pred.max(),101)
    axes[i].plot(y_true, y_pred, linestyle='', marker='o', markersize=0.4)
    axes[i].plot(x,x)
    axes[i].set_xlabel("PTS8.S1(CO) True Value")
    axes[i].set_ylabel("PTS8.S1(CO) Predicted Value")
    axes[i].set_title("Test %i" % i)
fig.suptitle("Relation between predicted and true value");